In [3]:


import cv2
import os
import socket
import sys
import pyttsx
import signal
import threading

reason = ""
inter = 0
reportTimeInterval = 30
frameTime = 0.3
steeringWheelHost = "raspberrypi.mshome.net"
steeringWheelPort = "5000"
DLCarHUDHost = "mattpi.mshome.net"
DLCarHUDPort = "4001"



from datetime import date
from datetime import timedelta
from time import sleep
import datetime

# Handle SIGINT
def handler(signum, frame):
    print('Signal handler called with signal', signum)
    global inter
    inter = 1
    
signal.signal(signal.SIGINT, handler)

date = date.today()
timeNow = datetime.datetime.now()

timeNow = timeNow.time()

timeString = str(timeNow)
dateString = str(date)
captureID = 'capture' + dateString[0:4] + dateString[5:7] + dateString[8:10] + timeString[0:2] + timeString[3:5]
print(captureID)

pictureDirectory = captureID + "/Pictures/"

if not os.path.exists(captureID):
    os.makedirs(captureID)
    
if not os.path.exists(pictureDirectory):
    os.makedirs(pictureDirectory)

def speakAsync(readOut):
    engine = pyttsx.init()
    engine.say(readOut)
    engine.runAndWait()
    print('done.')
    
    

# For sending request to retrieve accelerometer coordinates
def netcat(host, port, content):
    data = ""
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.settimeout(1.0)
    s.connect((host, int(port)))
    s.sendall(content.encode())
    s.shutdown(socket.SHUT_WR)
    while True:
        data = s.recv(8192)
        if not data:
            break
        data = data.decode("utf-8") 
        data = data.rstrip(' \0')
        return str(data)
    s.close()

camera_port = 0
if len(sys.argv) > 1:
    print(str(sys.argv))
    #camera_port = int(sys.argv[1])

#Number of frames to throw away while the camera adjusts to light levels
ramp_frames = 8000

# Now we can initialize the camera capture object with the cv2.VideoCapture class.
# All it needs is the index to a camera port.
try:
    camera = cv2.VideoCapture(camera_port)
    del(camera)
    camera = cv2.VideoCapture(camera_port)
except Warning:
   print("Failure")


global imageNum
imageNum = 0
# Captures a single image from the camera and returns it in PIL format

def get_image():
    global reason
    global imageNum
    # read is the easiest way to get a full image out of a VideoCapture object.
    imageCaptureSuccessful = 0
    imageWriteSuccessful = 0
    try:
        retval, im = camera.read()
        if(im.shape[0]<10):
            raise ValueError('Image shape is unexpected')
    except:
        reason = "Image capture failure"
        return 1
    try:
        file = "./" + str(captureID) + "/Pictures/"+str(captureID)+"image" + str(imageNum)+ ".png"
        imageNum = imageNum + 1
        cv2.imwrite(file, im)
    except:
        reason = "Image write failure"
        return 1
    try:
        steeringInfo = netcat(steeringWheelHost,steeringWheelPort, "request")
    except:
        reason = "Steering Wheel Failure"
        try:
            os.remove(file)
        except:
            print("File removal failed")
        return 1
    try:
        dlCarHUDInfo = netcat(DLCarHUDHost,DLCarHUDPort, "request")
    except:
        print("Obtaining DLCarHUD Data Failure")
        reason = "DLCarHUD Data Failure"
        try:
            os.remove(file)
        except:
            print("File removal failed")
            
        os.remove(file)
        return 1
    segments = dlCarHUDInfo.split(',')
    speed = segments[1]
    speed = (speed.split(':'))[1]
    speed = float(speed)
    if(speed < -1.0):
        print("Speed insufficient")
        reason = "Speed insufficient"
        try:
            os.remove(file)
        except:
            print("File removal failed")
        sleep(0.2)
        return -1
        
    dataFile = captureID + "/accelClass.txt"
    time = datetime.datetime.now().time()
    f = open(dataFile, 'a')
    f.write(dlCarHUDInfo)
    f.write(',')
    f.write(str(time))
    f.write(',')
    f.write(str(file))
    f.write(',')
    f.write(steeringInfo)  # python will convert \n to os.linesep
    f.write("\n")  # python will convert \n to os.linesep
    f.close()  # you can omit in most cases as the destructor will call it
    return 0

lasttime = datetime.datetime.now()
lastFrameTime = datetime.datetime.now()
failedFrameCount = 0
for i in range(ramp_frames):
    if inter == 1:
        break
    try:
        sleep(0.3)
        retVal = get_image()
        if(retVal==0):
            reason = "" #Clear reason flag
        else:
            print("Capture failed - trying again")
            if(reason==""):
                reason = "General Failure"
            i = i - 1
            failedFrameCount= failedFrameCount + 1
    except:
        print("Capture failed - trying again")
        reason = "General Failure"
        i = i - 1
        failedFrameCount= failedFrameCount + 1
    currenttime =  datetime.datetime.now()
    frameTimeDelta = abs(currenttime - lastFrameTime)
    lastFrameTime = currenttime
    print(frameTimeDelta)
    #if(frameTimeDelta.seconds<frameTime):
        #sleep(frameTime-frameTimeDelta.seconds)
    timeDelta = abs(currenttime - lasttime)
    if(timeDelta.seconds>reportTimeInterval):
        lasttime = datetime.datetime.now()
        if(len(reason) == 0):
            readOut = 'Successfully captured ' + str(i) + ' frames.'
            print(readOut)
        else:
            readOut = 'Capture failed due to ' + str(reason) + ".   The failed frame count is "+ str(failedFrameCount)
            print(readOut)
        t = threading.Thread(target=speakAsync, args=(readOut,))
        t.start()

# You'll want to release the camera, otherwise you won't be able to create a new
# capture object until your script exits
del(camera)



capture201708131455
['c:\\program files\\python35\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\ndmmc\\AppData\\Roaming\\jupyter\\runtime\\kernel-3c179567-f6c7-4510-8b53-35b076b9b770.json']
0:00:00.688577
0:00:00.370065
0:00:00.355471
0:00:00.355843
0:00:00.344436
0:00:00.408989
0:00:00.396618
0:00:00.376756
0:00:00.365805
0:00:00.463254
0:00:00.347993
0:00:00.381039
0:00:00.350980
0:00:00.368789
0:00:00.360796
0:00:00.384064
0:00:00.361517
0:00:00.355216
0:00:00.365450
0:00:00.380397
0:00:00.406330
0:00:00.703611
0:00:00.592335
0:00:00.367488
0:00:00.539566
0:00:00.458277
0:00:00.642918
0:00:00.618607
0:00:00.716146
0:00:00.450104
0:00:00.396457
0:00:00.592238
0:00:00.638842
0:00:00.690238
0:00:01.249557
0:00:00.509646
0:00:00.566517
0:00:00.509756
0:00:00.684560
0:00:00.544205
0:00:00.412882
0:00:01.342570
0:00:01.113214
0:00:01.226873
0:00:00.410178
0:00:01.094786
0:00:00.645608
0:00:00.457959
0:00:00.467825
0:00:00.612521
0:00:00.437080
0:00:00.569673
0:00:00.82029

Capture failed - trying again
0:00:00.314855
Capture failed - trying again
0:00:00.316763
Capture failed - trying again
0:00:00.314827
Capture failed - trying again
0:00:00.316484
Capture failed - trying again
0:00:00.315575
Capture failed - trying again
0:00:00.315228
Capture failed - trying again
0:00:00.315155
Capture failed - trying again
0:00:00.315023
Capture failed - trying again
0:00:00.314870
Capture failed - trying again
0:00:00.314822
Capture failed - trying again
0:00:00.319438
Capture failed - trying again
0:00:00.318002
Capture failed - trying again
0:00:00.318548
Capture failed - trying again
0:00:00.314234
Capture failed - trying again
0:00:00.315810
Capture failed - trying again
0:00:00.315875
Capture failed - trying again
0:00:00.318348
Capture failed - trying again
0:00:00.314590
Capture failed - trying again
0:00:00.315004
Capture failed - trying again
0:00:00.314040
Capture failed - trying again
0:00:00.314365
Capture failed - trying again
0:00:00.314310
Capture fa

Obtaining DLCarHUD Data Failure
Capture failed - trying again
0:00:01.277987
Obtaining DLCarHUD Data Failure
Capture failed - trying again
0:00:00.367761
Capture failed - trying again
0:00:01.374295
Capture failed due to Steering Wheel Failure.   The failed frame count is 285
Capture failed - trying again
0:00:01.351726
Capture failed - trying again
0:00:01.345802
Capture failed - trying again
0:00:01.522168
Obtaining DLCarHUD Data Failure
Capture failed - trying again
0:00:00.351936
Obtaining DLCarHUD Data Failure
Capture failed - trying again
0:00:00.549210
Obtaining DLCarHUD Data Failure
Capture failed - trying again
0:00:00.453060
done.
Obtaining DLCarHUD Data Failure
Capture failed - trying again
0:00:00.394853
Obtaining DLCarHUD Data Failure
Capture failed - trying again
0:00:00.404718
Obtaining DLCarHUD Data Failure
Capture failed - trying again
0:00:00.398049
Obtaining DLCarHUD Data Failure
Capture failed - trying again
0:00:00.368981
Obtaining DLCarHUD Data Failure
Capture fai

0:00:00.377177
0:00:01.054245
0:00:01.268577
0:00:00.396647
0:00:00.454550
0:00:00.419150
0:00:00.526545
0:00:00.616889
0:00:00.541049
0:00:00.442599
0:00:01.314573
Capture failed - trying again
0:00:01.420925
Capture failed - trying again
0:00:01.608931
0:00:00.589422
0:00:00.563789
0:00:00.750815
Successfully captured 559 frames.
0:00:01.245078
0:00:00.648649
0:00:00.631405
done.
0:00:01.442998
0:00:00.617397
0:00:00.581191
0:00:00.384284
0:00:00.395318
0:00:00.761249
0:00:00.537745
0:00:00.488128
0:00:00.436496
Capture failed - trying again
0:00:01.771262
Capture failed - trying again
0:00:01.314875
Capture failed - trying again
0:00:01.907939
0:00:00.433175
0:00:02.111686
0:00:00.790504
0:00:01.012226
Capture failed - trying again
0:00:01.317431
0:00:00.406710
0:00:00.467147
0:00:00.766910
0:00:01.054269
0:00:00.700576
0:00:00.879524
0:00:00.601410
Capture failed - trying again
0:00:01.528825
0:00:00.916610
0:00:00.777134
0:00:00.932738
0:00:00.467924
0:00:00.512739
0:00:00.434636


In [ ]:
import cv2
import os
import socket
import sys
import time
steeringWheelHost = "192.168.137.168"
steeringWheelPort = "5000"
# For sending request to retrieve accelerometer coordinates
def netcat(host, port, content):
    data = ""
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((host, int(port)))
    s.sendall(content.encode())
    s.shutdown(socket.SHUT_WR)
    while True:
        data = s.recv(8192)
        if not data:
            break
        data = data.decode("utf-8") 
        data = data.rstrip(' \0')
        return str(data)
    s.close()
while(1):
    steeringInfo = netcat(steeringWheelHost,steeringWheelPort, "request") 
    print(steeringInfo)
    time.sleep(0.1)


In [ ]:
dlCarHUDInfo = netcat(DLCarHUDHost,DLCarHUDPort, "request")
print(dlCarHUDInfo)